# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [5]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [7]:
# Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [9]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [10]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [11]:
# Train the model
normalized_input_model.fit(X_train, y_train, batch_size=32, epochs=150, validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 11857587160638051974756958208.0000 - mse: 11857587160638051974756958208.0000 - val_loss: 33800600967242535978139648.0000 - val_mse: 33800600967242535978139648.0000
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 19093974606730257191927808.0000 - mse: 19093974606730257191927808.0000 - val_loss: 8909338700969210525253632.0000 - val_mse: 8909338700969210525253632.0000
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 5032890902155939557670912.0000 - mse: 5032890902155939557670912.0000 - val_loss: 2348370968869314792259584.0000 - val_mse: 2348370968869314792259584.0000
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 1326595870678565494194176.0000 - mse: 1326595870678565494194176.0000 - val_loss: 618995773603260045197312.0000 - val_mse: 618995773603260045197312.0000
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 349670968

33/33 [==============================] - 0s 1ms/step - loss: 6178331136.0000 - mse: 6178331136.0000 - val_loss: 5265745408.0000 - val_mse: 5265745408.0000
Epoch 45/150
33/33 [==============================] - 0s 1ms/step - loss: 6177982976.0000 - mse: 6177982976.0000 - val_loss: 5266232832.0000 - val_mse: 5266232832.0000
Epoch 46/150
33/33 [==============================] - 0s 1ms/step - loss: 6178314240.0000 - mse: 6178314240.0000 - val_loss: 5262552576.0000 - val_mse: 5262552576.0000
Epoch 47/150
33/33 [==============================] - 0s 1ms/step - loss: 6177847808.0000 - mse: 6177847808.0000 - val_loss: 5264691712.0000 - val_mse: 5264691712.0000
Epoch 48/150
33/33 [==============================] - 0s 1ms/step - loss: 6179637248.0000 - mse: 6179637248.0000 - val_loss: 5268906496.0000 - val_mse: 5268906496.0000
Epoch 49/150
33/33 [==============================] - 0s 1ms/step - loss: 6178697216.0000 - mse: 6178697216.0000 - val_loss: 5271001600.0000 - val_mse: 5271001600.0000
Epoch

Epoch 93/150
33/33 [==============================] - 0s 1ms/step - loss: 6178921472.0000 - mse: 6178921472.0000 - val_loss: 5272135680.0000 - val_mse: 5272135680.0000
Epoch 94/150
33/33 [==============================] - 0s 1ms/step - loss: 6180460032.0000 - mse: 6180460032.0000 - val_loss: 5269123584.0000 - val_mse: 5269123584.0000
Epoch 95/150
33/33 [==============================] - 0s 1ms/step - loss: 6178398208.0000 - mse: 6178398208.0000 - val_loss: 5268559872.0000 - val_mse: 5268559872.0000
Epoch 96/150
33/33 [==============================] - 0s 1ms/step - loss: 6177445376.0000 - mse: 6177445376.0000 - val_loss: 5265399808.0000 - val_mse: 5265399808.0000
Epoch 97/150
33/33 [==============================] - 0s 1ms/step - loss: 6178640384.0000 - mse: 6178640384.0000 - val_loss: 5266864128.0000 - val_mse: 5266864128.0000
Epoch 98/150
33/33 [==============================] - 0s 1ms/step - loss: 6177804800.0000 - mse: 6177804800.0000 - val_loss: 5267643392.0000 - val_mse: 52676433

33/33 [==============================] - 0s 1ms/step - loss: 6180403712.0000 - mse: 6180403712.0000 - val_loss: 5262089728.0000 - val_mse: 5262089728.0000
Epoch 142/150
33/33 [==============================] - 0s 1ms/step - loss: 6180053504.0000 - mse: 6180053504.0000 - val_loss: 5266566656.0000 - val_mse: 5266566656.0000
Epoch 143/150
33/33 [==============================] - 0s 1ms/step - loss: 6179405312.0000 - mse: 6179405312.0000 - val_loss: 5268327936.0000 - val_mse: 5268327936.0000
Epoch 144/150
33/33 [==============================] - 0s 1ms/step - loss: 6178782720.0000 - mse: 6178782720.0000 - val_loss: 5269596160.0000 - val_mse: 5269596160.0000
Epoch 145/150
33/33 [==============================] - 0s 1ms/step - loss: 6179442176.0000 - mse: 6179442176.0000 - val_loss: 5264821248.0000 - val_mse: 5264821248.0000
Epoch 146/150
33/33 [==============================] - 0s 1ms/step - loss: 6178925568.0000 - mse: 6178925568.0000 - val_loss: 5268453376.0000 - val_mse: 5268453376.0000


> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [12]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [13]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, y_train_scaled, batch_size=32, epochs=150, validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.4161 - mse: 0.4161 - val_loss: 0.2036 - val_mse: 0.2036
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2262 - mse: 0.2262 - val_loss: 0.1706 - val_mse: 0.1706
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1938 - mse: 0.1938 - val_loss: 0.1518 - val_mse: 0.1518
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1707 - mse: 0.1707 - val_loss: 0.1475 - val_mse: 0.1475
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1532 - mse: 0.1532 - val_loss: 0.1341 - val_mse: 0.1341
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1412 - mse: 0.1412 - val_loss: 0.1396 - val_mse: 0.1396
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1329 - mse: 0.1329 - val_loss: 0.1270 - val_mse: 0.1270
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1227 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0239 - mse: 0.0239 - val_loss: 0.1078 - val_mse: 0.1078
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0237 - mse: 0.0237 - val_loss: 0.1066 - val_mse: 0.1066
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0228 - mse: 0.0228 - val_loss: 0.1046 - val_mse: 0.1046
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0226 - mse: 0.0226 - val_loss: 0.1036 - val_mse: 0.1036
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0221 - mse: 0.0221 - val_loss: 0.1037 - val_mse: 0.1037
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0219 - mse: 0.0219 - val_loss: 0.1053 - val_mse: 0.1053
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0217 - mse: 0.0217 - val_loss: 0.1051 - val_mse: 0.1051
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0211 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1125 - val_mse: 0.1125
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1136 - val_mse: 0.1136
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0107 - mse: 0.0107 - val_loss: 0.1125 - val_mse: 0.1125
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0107 - mse: 0.0107 - val_loss: 0.1124 - val_mse: 0.1124
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0104 - mse: 0.0104 - val_loss: 0.1129 - val_mse: 0.1129
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0104 - mse: 0.0104 - val_loss: 0.1126 - val_mse: 0.1126
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1124 - val_mse: 0.1124
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [14]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 515us/step - loss: 0.0084 - mse: 0.0084


[0.008398078382015228, 0.008398078382015228]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [15]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 555us/step - loss: 0.1138 - mse: 0.1138


[0.11384868621826172, 0.11384868621826172]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [16]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

26515.23623892974

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [17]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu', input_shape=(n_features), kernel_initializer='he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 
# Train the model
he_model.fit(X_train, y_train_scaled, batch_size=32, epochs=150, validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.5087 - mse: 0.5087 - val_loss: 0.2108 - val_mse: 0.2108
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2557 - mse: 0.2557 - val_loss: 0.1993 - val_mse: 0.1993
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2035 - mse: 0.2035 - val_loss: 0.1557 - val_mse: 0.1557
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1800 - mse: 0.1800 - val_loss: 0.1488 - val_mse: 0.1488
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1586 - mse: 0.1586 - val_loss: 0.1689 - val_mse: 0.1689
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1505 - mse: 0.1505 - val_loss: 0.1423 - val_mse: 0.1423
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1308 - mse: 0.1308 - val_loss: 0.1414 - val_mse: 0.1414
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1213 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0227 - mse: 0.0227 - val_loss: 0.1495 - val_mse: 0.1495
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0222 - mse: 0.0222 - val_loss: 0.1490 - val_mse: 0.1490
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0220 - mse: 0.0220 - val_loss: 0.1476 - val_mse: 0.1476
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0216 - mse: 0.0216 - val_loss: 0.1489 - val_mse: 0.1489
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0213 - mse: 0.0213 - val_loss: 0.1494 - val_mse: 0.1494
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0210 - mse: 0.0210 - val_loss: 0.1499 - val_mse: 0.1499
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0206 - mse: 0.0206 - val_loss: 0.1496 - val_mse: 0.1496
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0203 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1550 - val_mse: 0.1550
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1551 - val_mse: 0.1551
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1553 - val_mse: 0.1553
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1556 - val_mse: 0.1556
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1554 - val_mse: 0.1554
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.1556 - val_mse: 0.1556
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.1548 - val_mse: 0.1548
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [18]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 515us/step - loss: 0.0076 - mse: 0.0076


[0.007610849570482969, 0.007610849570482969]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [19]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 555us/step - loss: 0.1571 - mse: 0.1571


[0.1570717990398407, 0.1570717990398407]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [20]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=n_features))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.4555 - mse: 0.4555 - val_loss: 0.1649 - val_mse: 0.1649
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1997 - mse: 0.1997 - val_loss: 0.1462 - val_mse: 0.1462
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1665 - mse: 0.1665 - val_loss: 0.1351 - val_mse: 0.1351
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1427 - mse: 0.1427 - val_loss: 0.1241 - val_mse: 0.1241
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1284 - mse: 0.1284 - val_loss: 0.1356 - val_mse: 0.1356
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1187 - mse: 0.1187 - val_loss: 0.1208 - val_mse: 0.1208
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1098 - mse: 0.1098 - val_loss: 0.1188 - val_mse: 0.1188
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0984 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0217 - mse: 0.0217 - val_loss: 0.1219 - val_mse: 0.1219
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0207 - mse: 0.0207 - val_loss: 0.1231 - val_mse: 0.1231
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0200 - mse: 0.0200 - val_loss: 0.1261 - val_mse: 0.1261
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0205 - mse: 0.0205 - val_loss: 0.1269 - val_mse: 0.1269
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0200 - mse: 0.0200 - val_loss: 0.1235 - val_mse: 0.1235
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0194 - mse: 0.0194 - val_loss: 0.1238 - val_mse: 0.1238
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0193 - mse: 0.0193 - val_loss: 0.1250 - val_mse: 0.1250
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0192 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.1324 - val_mse: 0.1324
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1323 - val_mse: 0.1323
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1345 - val_mse: 0.1345
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1338 - val_mse: 0.1338
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1327 - val_mse: 0.1327
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.1321 - val_mse: 0.1321
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.1326 - val_mse: 0.1326
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 485us/step - loss: 0.0079 - mse: 0.0079


[0.007906933315098286, 0.007906933315098286]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on validate data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 485us/step - loss: 0.0079 - mse: 0.0079


[0.007906933315098286, 0.007906933315098286]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [23]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
rmsprop_model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])
# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.3609 - mse: 0.3609 - val_loss: 0.1526 - val_mse: 0.1526
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2002 - mse: 0.2002 - val_loss: 0.1280 - val_mse: 0.1280
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1461 - mse: 0.1461 - val_loss: 0.1196 - val_mse: 0.1196
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1061 - mse: 0.1061 - val_loss: 0.1205 - val_mse: 0.1205
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0864 - mse: 0.0864 - val_loss: 0.1336 - val_mse: 0.1336
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0730 - mse: 0.0730 - val_loss: 0.1407 - val_mse: 0.1407
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0619 - mse: 0.0619 - val_loss: 0.1525 - val_mse: 0.1525
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0537 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1104 - val_mse: 0.1104
Epoch 62/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0085 - mse: 0.0085 - val_loss: 0.1344 - val_mse: 0.1344
Epoch 63/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0081 - mse: 0.0081 - val_loss: 0.1207 - val_mse: 0.1207
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0089 - mse: 0.0089 - val_loss: 0.1128 - val_mse: 0.1128
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0067 - mse: 0.0067 - val_loss: 0.1223 - val_mse: 0.1223
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0067 - mse: 0.0067 - val_loss: 0.1124 - val_mse: 0.1124
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0083 - mse: 0.0083 - val_loss: 0.1115 - val_mse: 0.1115
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0112 - mse: 0

Epoch 123/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0064 - mse: 0.0064 - val_loss: 0.1069 - val_mse: 0.1069
Epoch 124/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0051 - mse: 0.0051 - val_loss: 0.1055 - val_mse: 0.1055
Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0041 - mse: 0.0041 - val_loss: 0.0964 - val_mse: 0.0964
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0043 - mse: 0.0043 - val_loss: 0.1005 - val_mse: 0.1005
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0053 - mse: 0.0053 - val_loss: 0.1109 - val_mse: 0.1109
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0053 - mse: 0.0053 - val_loss: 0.1011 - val_mse: 0.1011
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0053 - mse: 0.0053 - val_loss: 0.1014 - val_mse: 0.1014
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 545us/step - loss: 0.0037 - mse: 0.0037


[0.003693983191624284, 0.003693983191624284]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [26]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 555us/step - loss: 0.0968 - mse: 0.0968


[0.09675369411706924, 0.09675369411706924]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [28]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
adam_model.compile(optimizer='Adam', loss='mse', metrics=['mse'])
# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.3385 - mse: 0.3385 - val_loss: 0.1138 - val_mse: 0.1138
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1412 - mse: 0.1412 - val_loss: 0.0978 - val_mse: 0.0978
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1092 - mse: 0.1092 - val_loss: 0.1170 - val_mse: 0.1170
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0861 - mse: 0.0861 - val_loss: 0.0967 - val_mse: 0.0967
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0684 - mse: 0.0684 - val_loss: 0.0885 - val_mse: 0.0885
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0522 - mse: 0.0522 - val_loss: 0.0959 - val_mse: 0.0959
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0456 - mse: 0.0456 - val_loss: 0.0982 - val_mse: 0.0982
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0393 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0034 - mse: 0.0034 - val_loss: 0.1010 - val_mse: 0.1010
Epoch 62/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0020 - mse: 0.0020 - val_loss: 0.0981 - val_mse: 0.0981
Epoch 63/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.1013 - val_mse: 0.1013
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 7.4150e-04 - mse: 7.4150e-04 - val_loss: 0.1002 - val_mse: 0.1002
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 6.7421e-04 - mse: 6.7421e-04 - val_loss: 0.1019 - val_mse: 0.1019
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 5.5806e-04 - mse: 5.5806e-04 - val_loss: 0.0992 - val_mse: 0.0992
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 3.6492e-04 - mse: 3.6492e-04 - val_loss: 0.1008 - val_mse: 0.1008
Epoch 68/150
33/33 [==============================] - 0s 

Epoch 121/150
33/33 [==============================] - 0s 1ms/step - loss: 1.9931e-04 - mse: 1.9931e-04 - val_loss: 0.0958 - val_mse: 0.0958
Epoch 122/150
33/33 [==============================] - 0s 1ms/step - loss: 2.4696e-04 - mse: 2.4696e-04 - val_loss: 0.0930 - val_mse: 0.0930
Epoch 123/150
33/33 [==============================] - 0s 1ms/step - loss: 3.5477e-04 - mse: 3.5477e-04 - val_loss: 0.0961 - val_mse: 0.0961
Epoch 124/150
33/33 [==============================] - 0s 1ms/step - loss: 6.3548e-04 - mse: 6.3548e-04 - val_loss: 0.0919 - val_mse: 0.0919
Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0932 - val_mse: 0.0932
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0024 - mse: 0.0024 - val_loss: 0.0927 - val_mse: 0.0927
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0035 - mse: 0.0035 - val_loss: 0.0977 - val_mse: 0.0977
Epoch 128/150
33/33 [================

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [29]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 545us/step - loss: 7.3673e-04 - mse: 7.3673e-04


[0.0007367270300164819, 0.0007367270300164819]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [30]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 777us/step - loss: 0.0912 - mse: 0.0912


[0.0912499949336052, 0.0912499949336052]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [31]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 800us/step - loss: 0.1444 - mse: 0.1444


[0.14440803229808807, 0.14440803229808807]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [32]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

29862.567972514847

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.